In [25]:
#---------------
# import modules
#---------------

import numpy as np
import cv2
import joblib as jb
import pickle as pk
import matplotlib.pyplot as plt
import mahotas

from os import listdir
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import Isomap
from sklearn.model_selection import train_test_split
from collections import Counter

from sklearn.neural_network import MLPClassifier
from neuralNetwork.perceptron_qpso import perceptron 
from neuralNetwork.perceptron_pso import perceptron_pso
from neuralNetwork.perceptron_pso_bound import perceptron_pso_bound  

In [ ]:
#----------------
#Global variables
#----------------

root_path = './dataset2/'
#train_path = 'seg_train/seg_train/'
#test_path = 'seg_test/seg_test/'
#pred_path = 'seg_pred/seg_pred/'

# Dataset classes: buildings, forest, glacier, mountain, sea, street
data_clases = np.array(['cloudy', 
						'rain', 
						'shine', 
						'sunrise'])
IMG_HEIGHT = 150
IMG_WIDTH = 150

#---------------------------------
# Extract Global features
#---------------------------------
bins = 4 

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

#---------------------------------
# prepare training and testing set
#---------------------------------

X_train_aux = []
y_train_aux = []

for c in range(len(data_clases)):
	for filename in listdir(root_path + data_clases[c] + '/'):
		# load image
		img_data = cv2.imread(root_path + data_clases[c] + '/' + filename) 
		img_data = cv2.resize(img_data, (IMG_HEIGHT, IMG_WIDTH))
        # Global Feature extraction
		fv_hu_moments = fd_hu_moments(img_data)
		#print(fv_hu_moments.shape)
		fv_haralick = fd_haralick(img_data)
		#print(fv_haralick.shape)        
		fv_histogram  = fd_histogram(img_data)
		#print(fv_histogram.shape)
		# Concatenate global features
		global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
		X_train_aux.append(global_feature)
		y_train_aux.append(c)
		print('> loaded %s %s' % (filename, img_data.shape))
		print('> global features %s %s' % (filename, global_feature.shape))

# convert to array
X_train_aux = np.array(X_train_aux)
y_train_aux = np.array(y_train_aux)

# normalize data
#X_train_aux = X_train_aux / 255

print('X_train shape: ', X_train_aux.shape)
print('y_train shape: ', y_train_aux.shape)

In [27]:
n_particulas = 100
max_iter = 1000
n_training = 10

In [28]:
t = MinMaxScaler()
t.fit(X_train_aux)
X_train_aux = t.transform(X_train_aux)

In [29]:
embedding = Isomap(n_components=42, n_neighbors=100)
X_transformed = embedding.fit_transform(X_train_aux)
print ('Shape of X_train transformed: ', X_transformed.shape)

Shape of X_train transformed:  (1122, 42)


In [30]:
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_transformed, y_train_aux, test_size=0.3, random_state=100)
print(f"Training target statistics: {Counter(y_train_bal)}")
print(f"Testing target statistics: {Counter(y_test_bal)}")

Training target statistics: Counter({3: 253, 0: 196, 2: 179, 1: 157})
Testing target statistics: Counter({0: 104, 3: 104, 2: 72, 1: 57})


In [ ]:
X_sample = len(X_train_bal)
X_input = len(X_train_bal[1])
X_class = len(np.unique(y_train_bal))

X_train = X_train_bal
y_train = y_train_bal
X_test = X_test_bal
y_test = y_test_bal

#---------------------------------------
# variable initialize for qpso algorithm
#---------------------------------------
beta = 1.13 #1.13

gBest_value = []
gBest = []
cost_test = []
metric_train = []

for i in range(n_training):
    # load perceptron
    nn = perceptron(X_sample, X_input, X_class)
    gBest.append( nn.train(X_train, y_train, beta, n_particulas, max_iter) )
    gBest_value.append(nn.g_best_value)
    metric_train.append(nn.g_best_value_iter)
    y_test_pred = np.argmax(nn.forward(X_test, gBest[i]), axis=1)
    cost_test.append(mean_squared_error(y_test, y_test_pred))
    print ('Test prediction cost with training: ', cost_test[i]) 
    min_cost = min(cost_test)
    print ("Min test prediction cost: ", min_cost)
    if  cost_test[i] <= min_cost:       
        np.save("mcw_gBest_113_100_1000_42_qdpso.npy", gBest[i])
        np.save("mcw_gBestIter_113_100_1000_42_qdpso.npy", nn.g_best_value_iter)
        np.save("mcw_avgBest_113_100_1000_42_qdpso.npy", nn.avg_best_value)
        np.save("mcw_stdBest_113_100_1000_42_qdpso.npy", nn.std_best_value)
        
print("=====================================================================")
print("=====================================================================")
print("Save train metric ....")
np.save("mcw_metric_113_100_1000_42_qdpso.npy", metric_train)
print("The best training is in iteration: ", cost_test.index(min(cost_test)))
print("The golbal best value is: ", gBest_value[cost_test.index(min(cost_test))])
print("Test prediction cost: ", min(cost_test))
print("=====================================================================")
print("=====================================================================")     

In [38]:
model_load = np.load('mcw_gBest_113_100_1000_42_qdpso.npy')

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.2789317507418398  ACC score:  0.8724035608308606
MSE:  0.05222929936305733  ACC score:  0.9821656050955414


In [ ]:
#---------------------------------------
# variable initialize for pso algorithm
#---------------------------------------
opt = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

gBest_value = []
gBest = []
cost_test = []
metric_train = []


for i in range(n_training):
    # load perceptron
    nn = perceptron_pso(X_sample, X_input, X_class)
    gBest.append( nn.train(X_train, y_train, opt, n_particulas, max_iter) )
    gBest_value.append(nn.best_cost)
    metric_train.append(nn.h_cost)
    y_test_pred = np.argmax(nn.forward(X_test, gBest[i]), axis=1)
    cost_test.append(mean_squared_error(y_test, y_test_pred))
    print ('Test prediction cost with training: ', cost_test[i]) 
    min_cost = min(cost_test)
    print ("Min test prediction cost: ", min_cost)
    if  cost_test[i] <= min_cost: 
        np.save("mcw_gBest_113_100_1000_42_pso.npy", gBest[i])
        np.save("mcw_gBestIter_113_100_1000_42_pso.npy", nn.h_cost)
        np.save("mcw_avgBest_113_100_1000_42_pso.npy", nn.avg_best_value)
        #np.save("pBest_113_100_100_2_42_1.npy", nn.h_pos)
        
print("=====================================================================")
print("=====================================================================")
print("Saving train metric .... ")
np.save("mcw_metric_113_100_1000_42_pso.npy", metric_train)
print("The best training is in iteration ", cost_test.index(min(cost_test)))
print("The golbal best value is: ", gBest_value[cost_test.index(min(cost_test))])
print("Test prediction cost: ", min(cost_test))
print("=====================================================================")
print("=====================================================================")

In [34]:
model_load = np.load('mcw_gBest_113_100_1000_42_pso.npy')

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.258160237388724  ACC score:  0.8694362017804155
MSE:  0.2267515923566879  ACC score:  0.9299363057324841


In [ ]:
#---------------------------------------
# variable initialize for pso algorithm
#---------------------------------------
opt = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

gBest_value = []
gBest = []
cost_test = []
metric_train = []


for i in range(n_training):
    # load perceptron
    nn = perceptron_pso_bound(X_sample, X_input, X_class)
    gBest.append( nn.train(X_train, y_train, opt, n_particulas, max_iter) )
    gBest_value.append(nn.best_cost)
    metric_train.append(nn.h_cost)
    y_test_pred = np.argmax(nn.forward(X_test, gBest[i]), axis=1)
    cost_test.append(mean_squared_error(y_test, y_test_pred))
    print ('Test prediction cost with training: ', cost_test[i]) 
    min_cost = min(cost_test)
    print ("Min test prediction cost: ", min_cost)
    if  cost_test[i] <= min_cost: 
        np.save("mcw_gBest_113_100_1000_42_pso_bound.npy", gBest[i])
        np.save("mcw_gBestIter_113_100_1000_42_pso_bound.npy", nn.h_cost)
        np.save("mcw_avgBest_113_100_1000_42_pso_bound.npy", nn.avg_best_value)
        #np.save("pBest_113_100_100_2_42_1.npy", nn.h_pos)
        
print("=====================================================================")
print("=====================================================================")
print("Saving train metric .... ")
np.save("mcw_metric_113_100_1000_42_pso_bound.npy", metric_train)
print("The best training is in iteration ", cost_test.index(min(cost_test)))
print("The golbal best value is: ", gBest_value[cost_test.index(min(cost_test))])
print("Test prediction cost: ", min(cost_test))
print("=====================================================================")
print("=====================================================================")

In [36]:
model_load = np.load('mcw_gBest_113_100_1000_42_pso_bound.npy')

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = np.argmax(nn.forward(X_test, model_load), axis=1)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)


y_train_pred_load = np.argmax(nn.forward(X_train, model_load), axis=1)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.4035608308605341  ACC score:  0.8249258160237388
MSE:  0.37070063694267513  ACC score:  0.8624203821656051
